# Azure DevOps

With Azure DevOps data scientists and application developers can work together to create and maintain AI-infused applications. Using a DevOps mindset is not new to software developers, who are used to running applications in production. However, data scientists in the past have often worked in silos and not followed best practices to facilitate the transition from development to production. With Azure DevOps data scientists can now develop with an eye toward production.

## Part 1: Getting started

This lab allows you to perform setup for building a **Continuous Integration/Continuous Deployment** pipeline related to Anomoly Detection and Predictive Maintenance.

### Pre-requisites

- Azure account
- Azure DevOps account
- Azure Machine Learning Service Workspace
- Basic knowledge of Python

After you launch your environment, follow the below steps:

### Azure Machine Learning Service Workspace

We will begin the lab by creating a new Machine Learning Service Workspace using Azure portal:

1. Login to Azure portal using the credentials provided with the environment.

2. Select **Create a Resource** and search the marketplace for **Machine Learning Service Workspace**.

![Market Place](../images/marketplace.png)

3. Select **Machine Learning Service Workspace** followed by **Create**:

![Create Workspace](../images/createWorkspace.png)

4. Populate the mandatory fields (Workspace name, Subscription, Resource group and Location):

![Workspace Fields](../images/workspaceFields.png)

### Sign in to Azure DevOps

Go to **https://dev.azure.com** and login using your Azure username and password. You will be asked to provide a name and email. An organization is created for you based on the name you provide. Within the organization, you will be asked to create a project. Name your project "ADPM" and click on **Create project**. With private projects, only people you give access to will be able to view this project. After logging in, you should see the below:

![Get Started](../images/getStarted.png)

### Create Service connection

The build pipeline for our project will need the proper permission settings so that it can create a remote compute target in Azure.  This can be done by setting up a **Service Connection** and authorizing the build pipeline to use this connection.

> If we didn't set up this **service connection**, we would have to interactively log into Azure (e.g. az login) everytime we run the build pipeline.

Setting up a service connection involves the following steps:
1. Click on **Project settings** in the bottom-left corner of your screen.
2. On the next page, search for menu section **Pipelines** and select **Service Connection**.
3. Create a **New service connection**, of type **Azure Resource Manager**.

![Get Started](../images/createServiceConnection.png)

4. On the page you are presented with, scroll down and click on the link saying **use the full version of the service connection dialog**.

![Get Started](../images/changeToFullVersionServiceConnection.png)

5. Begin filling out the full version of the form. All the information you need is provided in the lab setup page. If you closed this page, a link to it was emailed to you. Look for emails from **No Reply (CloudLabs) <noreply@cloudlabs.ai>**.

![Get Started](../images/fullDialogueServiceConnection.png "width=50")

   - **Important!** Set **connection name** to **serviceConnection** (careful about capitalization).
   - For **Service principal client ID** paste the field called **Application/Client Id** in the lab setup page.
   - Set **Scope level** to **Subscription**.
   - For **Subscription**, select the same which you have been using throughout the course. You may already have a compute target in there (e.g. "aml-copute") and a AML workspace.
   - **Important!** Leave **Resource Group** empty.
   - For **Service principal key** paste the filed called **Application Secret Key** in the lab setup page.
   - Allow all pipelines to use this connection.
   - Click on **Verify connection** to make sure the connection is valid and then click on **OK**.

### Repository

After you create your project in Azure DevOps, the next step is to clone our repository into your DevOps project. The simplest way is to go to **Repos > Files > Import** as shown below. Provide the clone url (https://github.com/azure/learnai-customai-airlift) in the wizard to import.

![import repository](../images/importGit.png)

You should now be able to see the git repo in your project.

## Part 2: Building a pipeline

Tha aim of this lab is to demonstrate how you can build a Continuous Integration/Continuous Deployment pipeline and kick it off when there is a new commit. This scenario is typically very common when a developer has updated the application part of the code repository or when the training script from a data scientist is updated.

### Hosted Agents

With Azure Pipelines, you've got a convenient option to build and deploy using a **Microsoft-hosted agent**. Each time you run a pipeline, you get a fresh virtual machine and maintenance/upgrades are taken care of. The virtual machine is discarded after one use. The Microsoft-hosted agent pool provides 5 virtual machine images to choose from:

- Ubuntu 16.04
- Visual Studio 2017 on Windows Server 2016
- macOS 10.13
- Windows Server 1803 (win1803) - for running Windows containers
- Visual Studio 2015 on Windows Server 2012R2

YAML-based pipelines will default to the Microsoft-hosted agent pool. You simply need to specify which virtual machine image you want to use.

### Code Repository

The repo is organized as follows:

```
    code
    code/testing/
    code/scoring/
    code/aml_config/
    data_sample
    azure-pipelines.yml
```

The `code` folder contains all the python scripts to build the pipeline. The testing and scoring scripts are located in `code/testing/` and `code/scoring/` respectively. The config files created by the scripts are stored in `code/aml_config/`.

Sample data is created in `data_sample` that is used for testing. `azure-pipelines.yml` file at the root of your repository contains the instructions for the pipeline.

## About the scripts

For the purpose of DevOps, it's best not to use a Notebook because it can be error-prone. Instead, we have all the code sitting in individual Python scripts. This means that if we used a Notebook to develop our scripts, like we did throughout this course, we have some work to do to refactor the code and turn it into a series of modular Python scripts. We would also add scripts for running various tests everytime our build is triggered, such as unit tests, integration tests, tests to measure **drift** (a degradation over time of the predictions returned by the model on incoming data), etc.

Let's take a look at the different scripts we have to deal with and what each does.

In [ ]:
# %load ./code/pipeline.py

############################### load required libraries

import os
import pandas as pd
import json

import azureml.core
from azureml.core import Workspace, Run, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.telemetry import set_diagnostics_collection
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, StepSequence

print("SDK Version:", azureml.core.VERSION)

############################### load workspace and create experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

experiment_name =  'aml-pipeline-cicd' # choose a name for experiment
project_folder = '.' # project folder

experiment = Experiment(ws, experiment_name)
print("Location:", ws.location)
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

set_diagnostics_collection(send_diagnostics=True)

############################### create a run config

cd = CondaDependencies.create(pip_packages=["azureml-sdk==1.0.17", "azureml-train-automl==1.0.17", "pyculiarity", "pytictoc", "cryptography==2.5", "pandas"])

amlcompute_run_config = RunConfiguration(framework = "python", conda_dependencies = cd)
amlcompute_run_config.environment.docker.enabled = False
amlcompute_run_config.environment.docker.gpu_support = False
amlcompute_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
amlcompute_run_config.environment.spark.precache_packages = False

############################### create AML compute

aml_compute_target = "aml-compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", 
                                                                idle_seconds_before_scaledown=1800, 
                                                                min_nodes = 0, 
                                                                max_nodes = 4)
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

############################### point to data and scripts

# we use this for exchanging data between pipeline steps
def_data_store = ws.get_default_datastore()

# get pointer to default blob store
def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

# Naming the intermediate data as anomaly data and assigning it to a variable
anomaly_data = PipelineData("anomaly_data", datastore = def_blob_store)
print("Anomaly data object created")

# model = PipelineData("model", datastore = def_data_store)
# print("Model data object created")

anom_detect = PythonScriptStep(name = "anomaly_detection",
                               # script_name="anom_detect.py",
                               script_name = "CICD/code/anom_detect.py",
                               arguments = ["--output_directory", anomaly_data],
                               outputs = [anomaly_data],
                               compute_target = aml_compute, 
                               source_directory = project_folder,
                               allow_reuse = True,
                               runconfig = amlcompute_run_config)
print("Anomaly Detection Step created.")

automl_step = PythonScriptStep(name = "automl_step",
                                # script_name = "automl_step.py", 
                                script_name = "CICD/code/automl_step.py", 
                                arguments = ["--input_directory", anomaly_data],
                                inputs = [anomaly_data],
                                # outputs = [model],
                                compute_target = aml_compute, 
                                source_directory = project_folder,
                                allow_reuse = True,
                                runconfig = amlcompute_run_config)

print("AutoML Training Step created.")

############################### set up, validate and run pipeline

steps = [anom_detect, automl_step]
print("Step lists created")

pipeline = Pipeline(workspace = ws, steps = steps)
print ("Pipeline is built")

pipeline.validate()
print("Pipeline validation complete")

pipeline_run = experiment.submit(pipeline) #, regenerate_outputs=True)
print("Pipeline is submitted for execution")

# Wait until the run finishes.
pipeline_run.wait_for_completion(show_output = False)
print("Pipeline run completed")

############################### upload artifacts to AML Workspace

# Download aml_config info and output of automl_step
def_data_store.download(target_path = '.',
                        prefix = 'aml_config',
                        show_progress = True,
                        overwrite = True)

def_data_store.download(target_path = '.',
                        prefix = 'outputs',
                        show_progress = True,
                        overwrite = True)
print("Updated aml_config and outputs folder")

model_fname = 'model.pkl'
model_path = os.path.join("outputs", model_fname)

# Upload the model file explicitly into artifacts (for CI/CD)
pipeline_run.upload_file(name = model_path, path_or_stream = model_path)
print('Uploaded the model {} to experiment {}'.format(model_fname, pipeline_run.experiment.name))


The script `pipeline.py` run `anom_detect.py` and `automl_step.py` in that order. Let's see what these two scripts contain.

In [ ]:
# %load ./code/anom_detect.py

In [ ]:
# %load ./code/automl_step.py

%load ./code/evaluate_model.py
%load ./code/register_model.py
%load ./code/create_scoring_image.py
%load ./code/deploy_aci.py
%load ./code/aci_service_test.py

%load ./code/deploy_aks.py
%load ./code/aks_service_test.py
%load ./code/data_prep.py
%load ./code/scoring/score.py
%load ./code/testing/data_test.py

In [ ]:
# %load ./azure-pipelines.yml
pool:
  vmImage: 'Ubuntu 16.04'
steps:
- task: UsePythonVersion@0
  inputs:
    versionSpec: 3.5
    architecture: 'x64'

- task: DownloadSecureFile@1
  inputs:
    name: configFile
    secureFile: config.json
- script: echo "Printing the secure file path" 
- script: cp $(Agent.TempDirectory)/config.json $(Build.SourcesDirectory)

- task: CondaEnvironment@1
  displayName: 'Create Conda Environment '
  inputs:
    createCustomEnvironment: true
    environmentName: azuremlsdk
    packageSpecs: 'python=3.6'
    updateConda: false
    createOptions: 'cython==0.29 urllib3<1.24'
- script: |
    pip install --user azureml-sdk==1.0.17 pandas
  displayName: 'Install prerequisites'

- task: AzureCLI@1
  displayName: 'Azure CLI CICD/code/pipeline.py'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python CICD/code/pipeline.py'

- task: AzureCLI@1
  displayName: 'Azure CLI CICD/code/evaluate_model.py'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python CICD/code/evaluate_model.py'

- task: AzureCLI@1
  displayName: 'Azure CLI CICD/code/register_model.py'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python CICD/code/register_model.py'

- task: AzureCLI@1
  displayName: 'Azure CLI CICD/code/create_scoring_image.py'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python CICD/code/create_scoring_image.py'

- task: AzureCLI@1
  displayName: 'Azure CLI CICD/code/deploy_aci.py'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python CICD/code/deploy_aci.py'
    
- task: AzureCLI@1
  displayName: 'Azure CLI CICD/code/aci_service_test.py'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python CICD/code/aci_service_test.py'
- script: |
    python CICD/code/testing/data_test.py CICD/data_sample/predmain_bad_schema.csv
  displayName: 'Test Schema'

### Creating a config file and uploading it as a Secure File

On your own labtop, create a file called `config.json` to capture the `subscription_id`, `resource_group`, `workspace_name` and `workspace_region`:

```
{
    "subscription_id": ".......",
    "resource_group": ".......",
    "workspace_name": ".......",
    "workspace_region": "......."
}
```

You can get all of the info from the Machine Learning Service Workspace created in the portal as shown below. **Attention:** For `workspace_region` use one word and all lowercase, e.g. `westus2`.

![ML Workspace](../images/configFileOnPortal.png)

It's not best practice to commit the above config information to your source repository. To address this, we can use the Secure Files library to store files such as signing certificates, Apple Provisioning Profiles, Android Keystore files, and SSH keys on the server without having to commit them to your source repository. Secure files are defined and managed in the Library tab in Azure Pipelines.

The contents of the secure files are encrypted and can only be used during the build or release pipeline by referencing them from a task. There's a size limit of 10 MB for each secure file.

#### Upload Secure File

1. Select **Pipelines**, **Library** and **Secure Files**, then **+Secure File** to upload `config.json` file.

![Upload Secure File](../images/uploadSecureFile.png)

2. Select the uploaded file `config.json` and ensure **Authorize for use in all pipelines** is ticked and click on **Save**.       

### Creating a build

Azure Pipelines allow you to build AI applications without needing to set up any infrastructure of your own. Python is preinstalled on Microsoft-hosted agents in Azure Pipelines. You can use Linux, macOS, or Windows agents to run your builds.

#### New Pipeline

1. To create a new pipeline, select **New pipeline** from the Pipelines blade:

    ![New Pipeline](../images/newPipeline.png)

2. You will be prompted with **Where is your code?**. Select **Azure Repos** followed by your repo.

3. Select **Run**. Once the agent is allocated, you'll start seeing the live logs of the build.

#### Notification

The summary and status of the build will be sent to the email registered (i.e. Azure login user). Login using the email registered at `www.office.com` to view the notification.

### Azure Pipelines with YAML

You can define your pipeline using a YAML file: `azure-pipelines.yml` alongside the rest of the code for your app. The big advantage of using YAML is that the pipeline is versioned with the code and follows the same branching structure. 

The basic steps include:

1. Configure Azure Pipelines to use your Git repo.
2. Edit your `azure-pipelines.yml` file to define your build.
3. Push your code to your version control repository which kicks off the default trigger to build and deploy.
4. Code is now updated, built, tested, and packaged. It can be deployed to any target.

![Pipelines-Image-Yam](../images/pipelines-image-yaml.png)


Open the yml file in the repo to understand the build steps.

### Creating test scripts

In this workshop, multiple tests are included:

1. A basic test script `code/testing/data_test.py` is provided to test the schema of the json data for prediction using sample data in `data_sample/predmain_bad_schema.csv`.

2. `code/aci_service_test.py` and `code/aks_service_test.py` to test deployment using ACI and AKS respectively.

#### Exercise

- Can you either extend `code/testing/data_test.py` or create a new one to check for the feature types? 

- `code/aci_service_test.py` and `code/aks_service_test.py` scripts check if you are getting scores from the deployed service. Can you check if you are getting the desired scores by modifying the scripts?

- Make sure `azure-pipelines.yml` captures the above changes

#### Build trigger (continuous deployment trigger)

Along with the time triggers, we cann can also create a release every time a new build is available.

1. Enable the *Continuous deployment trigger* and ensure *Enabled* is selected in the *Continuous deployment trigger* configuration as shown below:

![Release Build Trigger](../images/releaseBuildTrigger.png)

2. Populate the branch in *Build branch filters*. A release will be triggered only for a build that is from one of the branches populated. For example, selecting "master" will trigger a release for every build from the master branch.

#### Approvals

For the QC task, you will recieve an *Azure DevOps Notifaction* email to view approval. On selecting *View Approval*, you will be taken to the following page to approve/reject:

![Pending Approval](../images/pendingApproval.png)

There is also provision to include comments with approval/reject:

![Approval Comments](../images/approvalComments.png)

Once the post-deployment approvals are approved by the users chosen, the pipeline will be listed with a green tick next to QC under the list of release pipelines: 

![Release Passed](../images/releasePassed.png)

### Application Insights (optional)

For your convenience, Azure Application Insights is automatically added when you create the Azure Machine Learning workspace. In this section, we will look at how we can investigate the predictions from the service created using `Analytics`. Analytics is the powerful search and query tool of Application Insights. Analytics is a web tool so no setup is required.

Run the below script (after replacing `<scoring_url>` and `<key>`) locally to obtain the predictions. You can also change `input_j` to obtain different predictions.

```python
import requests
import json

input_j = [[1.92168882e+02, 5.82427351e+02, 2.09748253e+02, 4.32529303e+01, 1.52377597e+01, 5.37307613e+01, 1.15729573e+01, 4.27624778e+00, 1.68042813e+02, 4.61654301e+02, 1.03138200e+02, 4.08555785e+01, 1.80809993e+01, 4.85402042e+01, 1.09373285e+01, 4.18269355e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.07200000e+03, 5.64000000e+02, 2.22900000e+03, 9.84000000e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.03000000e+02, 6.63000000e+02, 3.18300000e+03, 3.03000000e+02, 5.34300000e+03, 4.26300000e+03, 6.88200000e+03, 1.02300000e+03, 1.80000000e+01]]

data = json.dumps({'data': input_j})
test_sample = bytes(data, encoding = 'utf8')

url = '<scoring_url>'
api_key = '<key>' 
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

resp = requests.post(url, test_sample, headers=headers)
print(resp.text)

```

1. From the Machine Learning Workspace in the portal, Select `Application Insights` in the overview tab:

![ML Workspace](../images/mlworkspace.png)

2. Select Analytics.

3. The predictions will be logged which can be queried in the Log Analytics page in the Azure portal as shown below. For example, to query `requests`, run the following query:

````
    requests
    | where timestamp > ago(3h)
````

![LogAnalytics Query](../images/logAnalyticsQuery.png)

### Data Changes

A data scientist may want to trigger the pipeline when new data is available. To illustrate this, a small incremental data is made available in `data_sample\telemetry_incremental.csv` which is picked up in the below code snippet of anom_detect.py:

````python
    print("Adding incremental data...")
    telemetry_incremental = pd.read_csv(os.path.join('data_sample/', 'telemetry_incremental.csv'))
    telemetry = telemetry.append(telemetry_incremental, ignore_index=True)
````

The data changes would cause a change in the model evaluation and if it's better than the baseline model, it would be propagated for deployment.